In [ ]:
import gensim, nltk, os

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

results_dict = dict()
# contains input corpus

# iterate through all files in the corpus folder
for root, dirs, files in os.walk('cleaned_texts_eng'):
    for file_name in files:
        print(file_name)
        with open(os.path.join(root, file_name),encoding='utf8') as rf:
            text = rf.read()
            token_list = pos_tag(word_tokenize(text))
            all_tokens = [lis[0] for lis in token_list]
            all_pos_tags = [lis[1] for lis in token_list]
                
        results_dict[file_name] = {"all_tokens": all_tokens, "all_pos_tags": all_pos_tags}

In [ ]:
import json

#with open("results_dict_48_originals.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)

with open("results_dict_48_originals.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [2]:
for file_name in results_dict:
    all_tokens, all_pos_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_pos_tags']
    noun_tokens = []
    for i in range(len(all_pos_tags)):
        if all_pos_tags[i].startswith("N"):
            noun_tokens.append(all_tokens[i])
    results_dict[file_name]["noun_tokens"] = noun_tokens

In [3]:
stopwords_file = 'stopwords1.txt'

with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().split())

for filename in results_dict:
    noun_token_list = results_dict[filename]['noun_tokens']
    all_words = [word for word in noun_token_list if len(word)>=2 and word not in stopwords]
    results_dict[filename]['cleaned_noun_tokens'] = all_words
#     print(all_words)
#     break

In [4]:
def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [5]:
texts = []
numbers = []
for item in results_dict.values():
    texts.append(item['cleaned_noun_tokens'])
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)
print(len(numbers))

[201, 808, 1220, 3194, 3316, 3415, 4353, 5320, 6608, 7326, 8089, 8721, 9408, 10131, 10313, 10329, 10563, 11154, 11422, 11803, 12089, 13043, 13589, 14110, 14359, 14372, 15007, 15351, 15605, 16083, 16232, 16288, 16400, 16699, 17280, 17453, 18025, 18093, 18412, 18473, 18872, 18936, 20542, 21461, 22653, 23552, 27001, 62017]
48


In [6]:
texts = []
numbers = []
textid2name = dict()
for key, item in results_dict.items():
    texts.append(item['cleaned_noun_tokens'])
    textid2name[len(texts)-1] = key
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)

def split_list(l, length=1000):
    target = len(l)//length + 1
    segment_length = round(len(l)/target)
    return split_into_blocks(l, segment_length)
new_texts = []

chunk2text = dict()
for i in range(len(texts)):
    text = texts[i]
    splitted_text = split_list(text)
    for j in range(len(splitted_text)):
        new_texts.append(splitted_text[j])
        chunk2text[len(new_texts)-1] = {"id of original doc": i, "name of original doc": textid2name[i], "chunk index": j, "total # chunks": len(splitted_text)}
print(len(new_texts))
texts = new_texts

[201, 808, 1220, 3194, 3316, 3415, 4353, 5320, 6608, 7326, 8089, 8721, 9408, 10131, 10313, 10329, 10563, 11154, 11422, 11803, 12089, 13043, 13589, 14110, 14359, 14372, 15007, 15351, 15605, 16083, 16232, 16288, 16400, 16699, 17280, 17453, 18025, 18093, 18412, 18473, 18872, 18936, 20542, 21461, 22653, 23552, 27001, 62017]
717


In [7]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [8]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(abc) for abc in texts]

In [ ]:
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=40,random_state = 1, iterations=1000, chunksize = 1000,eval_every = None)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
    print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
x = range(5,15)
# z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('主题数目')
plt.ylabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': y,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()


In [ ]:
def perplexity(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)


In [ ]:
x = range(1,21)
z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
#y = [coherence(i) for i in x]
plt.plot(x, z)
plt.xlabel('主题数目')
plt.zlabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': z,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()

In [ ]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics= 9, passes=100,random_state = 1, iterations=300, chunksize = 1000,eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

In [13]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics= 8, passes=100,random_state = 1, iterations=300, chunksize = 1000,eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

[(0, '0.014*"men" + 0.012*"man" + 0.011*"hand" + 0.008*"heart" + 0.008*"day" + 0.007*"thing" + 0.007*"face" + 0.007*"death" + 0.007*"eye" + 0.006*"god"'), (1, '0.007*"professor" + 0.007*"challenger" + 0.006*"lord" + 0.005*"day" + 0.004*"train" + 0.004*"time" + 0.004*"race" + 0.004*"club" + 0.004*"yacht" + 0.003*"world"'), (2, '0.012*"ticket" + 0.011*"silver" + 0.010*"dame" + 0.009*"squire" + 0.006*"doctor" + 0.004*"inn" + 0.004*"traveler" + 0.004*"professor" + 0.003*"telemark" + 0.003*"drammen"'), (3, '0.015*"mr" + 0.011*"cocoanut" + 0.005*"musket" + 0.004*"tent" + 0.004*"savage" + 0.004*"cask" + 0.004*"pig" + 0.004*"sir" + 0.003*"stockade" + 0.003*"tomorrow"'), (4, '0.014*"men" + 0.011*"time" + 0.008*"man" + 0.008*"day" + 0.007*"country" + 0.007*"way" + 0.006*"people" + 0.006*"part" + 0.006*"place" + 0.006*"house"'), (5, '0.014*"man" + 0.012*"hand" + 0.009*"woman" + 0.008*"face" + 0.008*"time" + 0.008*"people" + 0.008*"life" + 0.007*"heart" + 0.007*"eye" + 0.006*"death"'), (6, '0.014*

In [14]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.191388 -0.072967       1        1  26.584418
6     -0.110057  0.088626       2        1  22.555066
4     -0.119231  0.054115       3        1  20.083171
7     -0.139381  0.037171       4        1  17.479658
0     -0.100078 -0.166384       5        1   9.136486
1      0.107784  0.151077       6        1   3.185044
2      0.286469 -0.055773       7        1   0.638458
3      0.265881 -0.035865       8        1   0.337699, topic_info=          Term         Freq        Total Category  logprob  loglift
301         mr  2698.000000  2698.000000  Default  30.0000  30.0000
412       ship  2418.000000  2418.000000  Default  29.0000  29.0000
661     doctor   710.000000   710.000000  Default  28.0000  28.0000
36        boat  2123.000000  2123.000000  Default  27.0000  27.0000
405        sea  2725.000000  2725.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2718    mother     6.065106   883.570839   Topic8  -5.9213   0.7093
237      house     6.955051  2118.264123   Topic8  -5.7843  -0.0281
2258    season     4.610649   233.900035   Topic8  -6.1954   1.7642
655     dinner     3.716305   269.961255   Topic8  -6.4111   1.4052
615   children     3.290920   357.121378   Topic8  -6.5326   1.0039

[638 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
31128      1  0.990634        aca
3988       2  0.063602       aint
3988       4  0.926772       aint
14941      6  0.514172  allahabad
14941      8  0.342781  allahabad
...      ...       ...        ...
6213       1  0.979882   zululand
6213       3  0.013609   zululand
6214       1  0.973481      zulus
6214       3  0.015959      zulus
6043       1  0.971953    zuvendi

[1493 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 5, 8, 1, 2, 3, 4])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# Assuming vis_data is already prepared
# vis_data = gensimvis.prepare(lda_model, corpus, id2word)

# Save to a file
pyLDAvis.save_html(vis, 'lda_result_original48new.html')
